In [1]:
import function_front as f

In [2]:
%%time
df_train_1 = pd.read_csv("../save_csv/EHD_0_10.csv")
df_train_2 = pd.read_csv("../save_csv/EHD_10_50.csv")
df_train_3 = pd.read_csv("../save_csv/EHD_50_100.csv")
df_train_concat = pd.concat([df_train_1, df_train_2, df_train_3]).reset_index()
df_train_concat["pos_new"] = df_train_concat["full_review"].apply(f.kkma_pos)

Wall time: 12min 46s


In [3]:
pos_voca = f.make_pos_voca(df_train_concat, "EHD")

#### df_train없이 model과 pos_voca를 불러오면 된다

In [2]:
f.test_overview(49867193)

('도시바 칸비오 어드밴스 외장하드 DTC920', 279)

In [3]:
df_test = f.make_df_test(49867193)

도시바 칸비오 어드밴스 외장하드 DTC920's Review Total Count : 279
도시바 칸비오 어드밴스 외장하드 DTC920's Filtered Review Count : 130


In [4]:
valid_review_cnt = f.df_test_len(df_test)
valid_review_cnt

130

In [5]:
X_test, voca = f.Vectorizer_test(df_test, "EHD")
y_test = df_test["rating_filtered"]

In [6]:
pos_voca = f.read_pos_voca("EHD")

In [8]:
import pickle
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

model = pickle.load(open("save_model_pkl/model_EHD_0_100_13806.pkl", "rb"))
y_test_pred = model.predict(X_test)
print(classification_report(y_test, y_test_pred))

print(f1_score(y_test, y_test_pred, average='macro'))
# macro - 라벨마다의 데이터 크기를 고려하지 않은 단순평균
print(f1_score(y_test, y_test_pred, average='micro'))
# micro - 전체데이터의 true positives, false negatives, false positives를 계산
print(f1_score(y_test, y_test_pred, average='weighted'))
# weighted - 조화평균

             precision    recall  f1-score   support

          0       0.67      1.00      0.80         2
          1       1.00      0.99      1.00       128

avg / total       0.99      0.99      0.99       130

0.8980392156862745
0.9923076923076923
0.9930618401206637


In [9]:
f.neg_or_pos(y_test_pred)

(3, 127)

In [10]:
df_top_word_negative = f.top_word_negative(X_test, y_test_pred, df_test, pos_voca, voca)
df_top_word_negative.head()

,index,word,pos,count,review
0,2132,뽁,NNG,2,[. 뽁뽁이 한개 없이 상품만 덩그러니 오네요... 망가지면 어떡할라고... ]
1,4450,컴퓨터,NNG,1,[. 1테라짜리 다른거도 사용중인데 여기꺼는 이상하게 읽는데 너무 오래걸립니다....
2,2250,상단,NNG,1,"[. 귀찮아서 그냥 쓰지만, 물건이 올 때부터 상단에 기스가 나 있어요..그..."
3,3493,이상,NNG,1,[. 1테라짜리 다른거도 사용중인데 여기꺼는 이상하게 읽는데 너무 오래걸립니다....
4,2279,상품,NNG,1,[. 뽁뽁이 한개 없이 상품만 덩그러니 오네요... 망가지면 어떡할라고... ]


In [11]:
word_neg = f.top_word_word(df_top_word_negative)
count_neg = f.top_word_count(df_top_word_negative)
review_neg = f.top_word_review(df_top_word_negative)

In [12]:
display(word_neg)
display(count_neg)
display(review_neg)

['뽁',
 '컴퓨터',
 '상단',
 '이상',
 '상품',
 '때',
 '연결',
 '물건',
 '성능',
 '제가',
 '기분',
 '테스트',
 '기스',
 '테라',
 '위치',
 '불량품',
 '사용']

[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

[['.  뽁뽁이 한개 없이 상품만 덩그러니 오네요... 망가지면 어떡할라고...   '],
 ['.  1테라짜리 다른거도 사용중인데  여기꺼는 이상하게 읽는데 너무 오래걸립니다. 그리고 컴퓨터 조금만 위치바꿔도 연결 끊어져요....제가 불량품받은거라면 이거보고 말씀해주세요.   '],
 ['.  귀찮아서 그냥 쓰지만,    물건이 올 때부터 상단에 기스가 나 있어요..그것도 크게!    기분이 별로라 성능이 어떤지 테스트도 안해봤네요.   '],
 ['.  1테라짜리 다른거도 사용중인데  여기꺼는 이상하게 읽는데 너무 오래걸립니다. 그리고 컴퓨터 조금만 위치바꿔도 연결 끊어져요....제가 불량품받은거라면 이거보고 말씀해주세요.   '],
 ['.  뽁뽁이 한개 없이 상품만 덩그러니 오네요... 망가지면 어떡할라고...   '],
 ['.  귀찮아서 그냥 쓰지만,    물건이 올 때부터 상단에 기스가 나 있어요..그것도 크게!    기분이 별로라 성능이 어떤지 테스트도 안해봤네요.   '],
 ['.  1테라짜리 다른거도 사용중인데  여기꺼는 이상하게 읽는데 너무 오래걸립니다. 그리고 컴퓨터 조금만 위치바꿔도 연결 끊어져요....제가 불량품받은거라면 이거보고 말씀해주세요.   '],
 ['.  귀찮아서 그냥 쓰지만,    물건이 올 때부터 상단에 기스가 나 있어요..그것도 크게!    기분이 별로라 성능이 어떤지 테스트도 안해봤네요.   '],
 ['.  귀찮아서 그냥 쓰지만,    물건이 올 때부터 상단에 기스가 나 있어요..그것도 크게!    기분이 별로라 성능이 어떤지 테스트도 안해봤네요.   '],
 ['.  1테라짜리 다른거도 사용중인데  여기꺼는 이상하게 읽는데 너무 오래걸립니다. 그리고 컴퓨터 조금만 위치바꿔도 연결 끊어져요....제가 불량품받은거라면 이거보고 말씀해주세요.   '],
 ['.  귀찮아서 그냥 쓰지만,    물건이 올 때부터 상단에 기스가 나 있어요..그것도 크게!    기분이 별로라 성능이 어떤지 테스트도 안해봤네요.   '],


In [13]:
df_top_word_positive = f.top_word_positive(X_test, y_test_pred, df_test, pos_voca, voca)
df_top_word_positive.head()

,index,word,pos,count,review
0,4840,하드,NNG,34,[. 1) 포장 (☆☆☆☆☆) 엉망요. 비닐팩에 그냥 넣어져 왔음. 하물...
1,1775,배송,NNG,30,[. 1) 포장 (☆☆☆☆☆) 엉망요. 비닐팩에 그냥 넣어져 왔음. 하물...
2,3275,용량,NNG,26,[. 1) 포장 (☆☆☆☆☆) 엉망요. 비닐팩에 그냥 넣어져 왔음. 하물...
3,3237,외장,NNG,26,[. 1) 포장 (☆☆☆☆☆) 엉망요. 비닐팩에 그냥 넣어져 왔음. 하물...
4,1231,디자인,NNG,21,[. 기존에 쓰던 외장하드가 불안해서 새로샀는데 디자인도 너무 깔끔하고 작고 전...


In [14]:
df_top_word_positive["count"]

0     34
1     30
2     26
3     26
4     21
5     19
6     19
7     18
8     18
9     17
10    17
11    15
12    15
13    13
14    13
15    13
16    13
17    11
18    10
19    10
Name: count, dtype: int64

In [15]:
df_top_word_positive["count"].apply(lambda a : int(a))

0     34
1     30
2     26
3     26
4     21
5     19
6     19
7     18
8     18
9     17
10    17
11    15
12    15
13    13
14    13
15    13
16    13
17    11
18    10
19    10
Name: count, dtype: int64

In [16]:
word_pos = f.top_word_word(df_top_word_positive)
count_pos = f.top_word_count(df_top_word_positive)
review_pos = f.top_word_review(df_top_word_positive)

In [17]:
display(word_pos)
display(count_pos)
display(review_pos)

['하드',
 '배송',
 '용량',
 '외장',
 '디자인',
 '사용',
 '만족',
 '가격',
 '파우치',
 '속도',
 '구매',
 '필름',
 '케이스',
 '제품',
 '포장',
 '도시바',
 '보호',
 '상품',
 '크기',
 '색상']

[34,
 30,
 26,
 26,
 21,
 19,
 19,
 18,
 18,
 17,
 17,
 15,
 15,
 13,
 13,
 13,
 13,
 11,
 10,
 10]

[['.  1)  포장  (☆☆☆☆☆)  엉망요.  비닐팩에 그냥 넣어져 왔음.  하물며....안에 뽁뽁이 포장도 없음!! 헐...이거 외장하드에요!  이러시면 안되요. 쿠팡!  물리적 충격 걱정되시는분은 추천안할게요.    2018.6월 2번째 구매상품은 택배상자로 왔어요. 앞에 구매한건 비닐팩에 들었는데;;  택배포장안의 내용물은 뽁뽁이 포장없이 제품만 들어가있어요;;;  역시 뭔가 부족한 성의없는 포장...;; 앞에 구매했던 버건디 색이랑 같은 생각이 드네요  물리적 충격 걱정이신 분께 추천안합니다.      2)  배송  (★★★★☆)  빨라요. 역시 쿠팡물류창고에서 바로 나오는건가봐요.  나가는 순간에 그냥 툭 놔두고 가셨네요.. "잠시만요"해도 소용없네...  당장 급하게 필요하신분만 주문하시길...    3)  크기(★★★★☆)  작네요. 신용카드 2장 정도 크기 보다도 작음.    4)  무게(★★★★☆)  가벼워요. 본체가 플라스틱이라..  아이폰6S 보다 가벼워요.  간혹 들고 나갔을때 부담없는 무게네요.    5)  색상(★★★☆☆)  모르겠네요. 진한 버건디는 아닌 와인색?  나쁘지 않아요.    2018.6월 2번째 상품은 검정색으로 구매했어요.  이제보니 전 버건디색이 더 매력적인거 같아요.  좋아하는 색상으로 하면 될 것 같아요.      6)  저장방식(★★★★★)  PMR방식 으로 알고있어요.  포맷(기본값)한번하고 사용해야하네요.  포맷은 꼭 USB3.0포트에서 하시길 추천합니다.기억이 가물한대 2~3시간 정도 걸림.  USB 2.0포트에서는 하루가 걸릴 수도 있어요. 2.0 꼽았다가 놀래서 빼고 3.0포트에서 함.    7)  저장용량(★★★★★)  기타 유틸리티 프로그램 내장 안되어있어서  1.81tb로 90.5%의 용량 쓸수 있어서 좋아요.    8)  유틸리티(★★★★★)  보안 패스워드 설정같은 유틸리티들은  홈페이지에서 다운 가능해요.    9)  가격(★★★★☆)  업체마다 비슷비슷한대 WD제품군비해선 용량대비 몇천